<a href="https://colab.research.google.com/github/CumputingCloud-G15/HW3/blob/main/Panther-Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#@title info screen
import ipywidgets as widgets
from IPython.display import display, HTML

global page_style
global page_content

# Define the styling for the page
page_style = """
<style>
    .page-container {
        background-color: rgba(0, 0, 50, 0.7);
        padding: 50px;
        border-radius: 10px;
        width: 500px;
        height: 400px;
        margin: 0 auto;
        text-align: center;
        display: flex;
        flex-direction: column;
        justify-content: flex-start;
        align-items: center;
    }

    .logo-image {
        width: 90px;
        margin-top: 10px;
        opacity: 0.9;
        filter: drop-shadow(0px 2px 4px rgba(255, 255, 255, 0.8));
    }

    h1 {
        color: white;
        text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.8);
        font-family: "Arial", sans-serif;
        font-size: 24px;
        font-weight: bold;
        margin-bottom: 20px;
    }

    p {
        color: white;
        text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.8);
        font-family: "Arial", sans-serif;
        font-size: 16px;
        font-weight: bold;
        margin-bottom: 10px;
    }

    .back-button {
        position: fixed;
        bottom: 20px;
        left: 20px;
        background-color: #555555;
        color: white;
        border: none;
        padding: 10px 20px;
        font-size: 16px;
        border-radius: 5px;
        cursor: pointer;
    }

    .light-green-text {
        color: #80ff80;
        font-weight: bold;
    }

    .red-text {
        color: red;
        font-weight: bold;
    }
</style>
"""

# Create the page content
page_content = """
<div class="page-container">
    <img class="logo-image" src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/2048px-Python-logo-notext.svg.png">
    <h1>Welcome to OpenCloud Game !</h1>
    <p>Let's play a game!</p>
    <p>Answer Python-related questions to earn points.</p>
    <p class="light-green-text">Correct answers earn you <span class="light-green-text">1 point</span>.</p>
    <p class="red-text">Wrong answers earn you <span class="red-text">0 points</span>.</p>
    <p>Challenge yourself and see how many points you can score!</p>
    <p>Remember, practice makes perfect!</p>
</div>
"""
# line from page_content =     <button class="back-button" onclick="window.history.back()">Back</button>
# Display the page
#display(HTML(page_style + page_content))


In [ ]:
from ipywidgets.widgets.widget_color import Color
#@title Questions part
# adding indicator to keep score of current run
# making the exit button always on display


from firebase import firebase
from IPython.display import display, HTML, clear_output
import pprint
import json
import random
from math import ceil
from ipywidgets import HTML, RadioButtons, VBox, HBox, Layout
import ipywidgets as widgets
import matplotlib.pyplot as plt
import tempfile
import time
import datetime
import numpy as np

from google.colab.output import eval_js
eval_js('google.colab.output.setIframeHeight("750")')

FBconn = firebase.FirebaseApplication('https://tirgul6---cloud-default-rtdb.firebaseio.com/',None)

questions = FBconn.get('/Questions/', None)
#question_text = answers_box = input_widget = feedback_box = None
question_text = answers_box = info_button = feedback_box = goodbye_text = None
session_btn = None

i = 0
score = 0
total_questions = len(questions)
input_amount = 0
menu_text = None
page_container = None
spheres_html = None
logout_hbox = None
exit_box = None
result_box = None
correct_amount = []
answers = []
user_data = None
user_name = ""
session_id = None
pia_chart = None

continue_button = widgets.Button(description='Continue', button_style='success')

exit_button = widgets.Button(description='Exit current game', button_style='danger')
exit_button.add_class("exit_box")

display(HTML("""
  <style>
  .exit_box {
    /* padding: 35px */
  }
  </style>
"""))

logout_button = widgets.Button(description='Log Out', button_style='info')
motivational_phrases = [
    "Embrace practice, learn from mistakes, and improve your Python skills to become an expert.",
    "Trust the learning process, believe in yourself, and watch your Python accuracy grow.",
    "Refine your coding technique, stay dedicated, and see your Python accuracy soar.",
    "Every question answered brings you closer to Python proficiency; never give up!",
    "Your progress in Python is evident; strive for even greater accuracy with practice.",
    "Halfway to Python mastery; keep coding, exploring, and maintaining a positive mindset.",
    "Build on your Python foundation, address weaknesses, and improve accuracy.",
    "Impressive accuracy in Python; challenge yourself and aim higher.",
    "Your Python accuracy shines; keep pushing boundaries and honing your skills.",
    "Master Python, inspire others, and strive for perfection in every question."
]

def refresh_exit_button():
  global exit_box
  global exit_button
  if exit_box != None:
    exit_box.close()
  exit_button = widgets.Button(description='Exit current game',
                               button_style='danger')
  exit_button.on_click(on_exit_clicked)
  exit_box = widgets.VBox([exit_button],
                          layout=widgets.Layout(align_items='center',
                                                justify_content='center'))
  exit_box.add_class("exit_box")

  display(HTML("""
    <style>
    .exit_box {
      /* padding: 35px 0 */
    }
    </style>
  """))



# define a function for logging out from current user
def on_logout_clicked(b):
  close_widgets()
  global goodbye_text
  goodbye_text = widgets.HTML('<h3 style="color: lightgreen">Returning to log-in screen shortly</h3>')
  display(goodbye_text)
  time.sleep(5)
  goodbye_text.close()
  # instead of return we go to log-in screen
  Login()

# Define a function to ask a question
def ask_question(question):
  global question_text
  global i
  global answers
  global answers_box
  global exit_box
  global page_container
  global input_amount
  global question_counter
  refresh_exit_button()


  question_counter = widgets.HTML(f'<div><h1 style="text-align:center;">Question {i+1}/{input_amount}</h1></div>')
  question_counter.add_class("question_counter")

  # Display the question
  question_text = widgets.HTML(f'<div><h3 style="text-align:center;">{question["Question"]}</h3></div>')

  display(HTML("""
    <style>
    .question_counter {
      padding: 35px 0 10px
    }
    </style>
  """))

  # define a list of answer choices with indices
  answers = [[(question["Option1"], 0), (question["Option2"], 1)], [(question["Option3"], 2), (question["Option4"], 3)]]

    # create buttons for each answer choice
  buttons = [[widgets.Button(description=answers[row][col][0], button_style='') for row in range(2)] for col in range(2)]
  for j in range(2):
    for h in range(2):
      buttons[j][h].add_class('buttons_css')

  # set layout for each button
  for button_row in buttons:
    for button in button_row:
      button.layout.width = '550px'
      button.layout.height = 'auto'

  # add button click event handlers
  for col in buttons:
      for button in col:
          button.on_click(lambda b: on_button_click(b))

  # display the buttons
  answers_box = widgets.VBox([widgets.HBox([buttons[0][0], buttons[1][0]]), widgets.HBox([buttons[0][1], buttons[1][1]])])
  answers_box = widgets.VBox([answers_box], layout=widgets.Layout(align_items='center', justify_content='center'))
  answers_box.add_class("answers_box")
  display(HTML("""
    <style>
    .answers_box {
      padding: 5px 0 35px 0
    }
    </style>
  """))

  page_container = widgets.VBox([question_counter, question_text, answers_box, exit_box])
  display(page_container)
  page_container.add_class('body')


  # define a function to handle button clicks
  def on_button_click(button):
    global score
    global feedback_box
    global continue_button
    global input_amount
    global i
    global correct_amount
    global answers
    global exit_box
    global page_container
    # disable all buttons
    for col in buttons:
        for btn in col:
            btn.disabled = True
    # get the index of the clicked button
    col_idx = -1
    for col_index, col in enumerate(buttons):
        if button in col:
            col_idx = col_index
            break
    row_idx = col.index(button)
    button_index = answers[row_idx][col_idx][1]
    # color the buttons based on correctness
    correct_index = question['Correct']-1
    for col in buttons:
        for btn in col:
            if btn == button:
                # if I clicked the right answer -> paint my button green
                # add 1 to my score
                # set in correct_amount array that this question I got right
                if button_index == correct_index:
                    button.style.button_color = 'green'
                    score += 1
                    correct_amount.append(1)
                # else -> paint my button red
                # don't add 1 to my score
                # set in correct_amount array that this question I got wrong
                else:
                    button.style.button_color = 'red'
                    correct_amount.append(0)
            # in any case -> paint the correct button green
            elif answers[col.index(btn)][buttons.index(col)][1] == correct_index:
                btn.style.button_color = 'green'
    # Ask the player if they want to continue
    continue_text = "Continue"
    if i+1 == input_amount:
      continue_text = "View Results"
    continue_button = widgets.Button(description=continue_text, button_style='success')
    refresh_exit_button()
    continue_box = widgets.VBox([continue_button, exit_box],
                                layout=widgets.Layout(align_items='center',
                                                      justify_content='center'))
    # display(continue_button)
    continue_button.on_click(on_continue_clicked)
    #display(continue_box)
    page_container = widgets.VBox([continue_box])
    display(page_container)
    page_container.add_class('body')

def show_session():
  global user_data
  global user_name
  global motivational_phrases
  # calculating user total score
  dates = []
  scores = []
  totalAmounts = []

  for session in user_data["sessions"].values():
    #print(session)
    dates.append(session["created"])
    scores.append(session["score"])
    totalAmounts.append(session["total"])

  num_of_games_to_show = 10
  # calulate date of last 10 games
  x_representation = []
  win_rates = []
  i = 1
  number_backwards = -1 * num_of_games_to_show
  for score, total_amount in zip(scores, totalAmounts):
    x_representation.append(f"{i} ({score}/{total_amount})")
    current_win_rate = (score / total_amount) * 100
    win_rates.append(current_win_rate)
    i += 1

  # Plotting the bar graph
  fig, ax = plt.subplots()
  bar_width = 0.35

  # Create the bars with color based on the percentage
  bars = ax.bar(x_representation[number_backwards:], win_rates[number_backwards:], bar_width)

  # Adding labels and title
  plt.xlabel('Games (old - new)')
  plt.ylabel('Accuracy (%)')
  plt.title(f'Accuracy (last {num_of_games_to_show} games)', color='white')

  # Rotating x-axis labels for better visibility
  plt.xticks(rotation=45)

  # Set the labels' color to white
  ax.xaxis.label.set_color('white')
  ax.yaxis.label.set_color('white')
  ax.tick_params(axis='x', colors='white')
  ax.tick_params(axis='y', colors='white')
  ax.spines['top'].set_color('white')
  ax.spines['right'].set_color('white')
  ax.spines['bottom'].set_color('white')
  ax.spines['left'].set_color('white')

  # Displaying the graph
  #plt.show()

  # Save the pie chart as an image file with transparent background
  with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as tmpfile:
      image_path = tmpfile.name
      plt.savefig(image_path, bbox_inches='tight', facecolor='none', transparent=True)


  # Close the figure to release memory
  plt.close()

  # Display the image inside a widgets.VBox
  image_widget = widgets.Image(value=open(image_path, 'rb').read(), format='png')
  image_widget.add_class("image_widget")

  sum_games = len(totalAmounts)
  avg_win_rate = 0.0
  if (len(win_rates)) > 0:
    avg_win_rate = sum(win_rates) / len(win_rates)
    avg_win_rate = round(avg_win_rate, 2)

  # claclulate phrase
  phrase_index = 9
  if avg_win_rate < 100:
    phrase_index = int(avg_win_rate // 10)
  phrase = motivational_phrases[phrase_index]

  title = widgets.HTML(value=f'<div><h1>{user_name}`s Profile</h1></div>')
  title.add_class("title")

  phrase_text = widgets.HTML(value=f'<div>“{phrase}”</div>')
  phrase_text.add_class("phrase_text")

  total_games = widgets.HTML(value=f'<div><h2>Total games: {sum_games}</h2></div>')
  win_rate_txt = widgets.HTML(value=f'<div><h2>Current Accuracy: {avg_win_rate}%</h2></div>')
  status_text = widgets.VBox([total_games, win_rate_txt])

  split_line = widgets.HBox([])
  split_line.add_class("split_line")

  bar_box = widgets.HBox([image_widget, split_line, status_text],
                          layout=widgets.Layout(align_items='center',
                                                justify_content='center'))


  profile_box = widgets.VBox([title, phrase_text, bar_box],
                             layout=widgets.Layout(align_items='center',
                                                justify_content='center'))

  display(HTML("""
      <style>
      .title {
        text-decoration: underline;
        padding: 0 0 25px 0
      }
      .phrase_text {
        font-family: Arial, sans-serif;
        font-size: 26px;
        color: #fff;
        text-align: center;
        margin: 20px;
        text-shadow: 0 0 5px #ff9900, 0 0 10px #ff9900, 0 0 15px #ff9900;
        animation: shining 1s infinite;
      }
      .phrase_text::first-letter {
        font-size: 36px;
        font-weight: bold;
      }
      @keyframes shining {
        0% { text-shadow: 0 0 5px #ff9900, 0 0 10px #ff9900, 0 0 15px #ff9900; }
        50% { text-shadow: 0 0 10px #ff9900, 0 0 15px #ff9900, 0 0 20px #ff9900; }
        100% { text-shadow: 0 0 5px #ff9900, 0 0 10px #ff9900, 0 0 15px #ff9900; }
      }
      .split_line {
        border-left: 6px solid #1f77b4;
        height: 100px;
        margin-right: 10px
      }
      .image_widget {
        backdrop-filter: blur(10px);
        margin-right: 60px
      }
      </style>
  """))

  return profile_box

# define a function that gets the amount of questions to display
def start_game(amount):
  global input_amount
  global menu_text
  global page_container
  global info_button
  global session_btn
  global session_id
  import uuid
  session_id = str(uuid.uuid1())

  # set global ver input_amount as that amount
  input_amount = amount

  # close menu and buttons for amounts
  menu_text.close()
  info_button.close()
  session_btn.close()
  page_container.close()

  # Shuffle the questions
  random.shuffle(questions)

  # Play the game
  ask_question(questions[i])

def open_sessionGraph(b):
  global logout_button
  global logout_hbox
  global user_data
  global user_name

  close_widgets()

  #logout_button = widgets.Button(description='Log Out', button_style='primary')
  logout_button = widgets.Button(description='Log Out', layout=widgets.Layout(width='75px'))#, button_style='primary')
  logout_button.add_class('logoutBtn_css')
  logged_in_user = widgets.HTML(value=f'<div><h3>Hello {user_name} ({user_data["points"]} points earned)</h3></div>')
  logout_hbox = widgets.HBox([logged_in_user, logout_button],
                             layout=widgets.Layout(justify_content='flex-end',
                                                   align_items='center'))
  display(logout_hbox)
  logout_button.on_click(on_logout_clicked)
  bar_box = show_session()

  display(bar_box)
  back_button = widgets.Button(description='Back to menu', button_style='')
  back_button.add_class('logoutBtn_css')
  display(back_button)

  back_button.on_click(lambda b: openMenu())

  display(HTML("""
      <style>
      body {
          background-image: url('https://cutewallpaper.org/21/quiz-background/General-knowledge-quizzes-What-do-you-know.jpg');
          background-size: 1820px 620px;
        }
      .buttons_css {
        border-radius: 10px;
        background-color: dodgerblue;
      }
      .logoutBtn_css {
        border-radius: 10px;
        background-color: cornsilk;
        color: black;
      }
      .infoBtn_css {
        border-radius: 10px;
        background-color: cornsilk;
        color: black;
        font-size: 14px;
      }
      </style>
  """))

def open_info():
  global logout_button
  global logout_hbox
  global user_data
  global user_name
  global page_content
  global page_style

  close_widgets()

  #logout_button = widgets.Button(description='Log Out', button_style='primary')
  logout_button = widgets.Button(description='Log Out', layout=widgets.Layout(width='75px'))#, button_style='primary')
  logout_button.add_class('logoutBtn_css')
  logged_in_user = widgets.HTML(value=f'<div><h3>Hello {user_name} ({user_data["points"]} points earned)</h3></div>')
  logout_hbox = widgets.HBox([logged_in_user, logout_button],
                             layout=widgets.Layout(justify_content='flex-end',
                                                   align_items='center'))
  display(logout_hbox)
  logout_button.on_click(on_logout_clicked)
  display(HTML(page_style + page_content))
  back_button = widgets.Button(description='Back to menu', button_style='')
  back_button.add_class('logoutBtn_css')
  display(back_button)

  back_button.on_click(lambda b: openMenu())

  display(HTML("""
        <style>
        body {
            background-image: url('https://cutewallpaper.org/21/quiz-background/General-knowledge-quizzes-What-do-you-know.jpg');
            background-size: 1820px 620px;
          }
        .buttons_css {
          border-radius: 10px;
          background-color: dodgerblue;
        }
        .logoutBtn_css {
          border-radius: 10px;
          background-color: cornsilk;
          color: black;
        }
        .infoBtn_css {
          border-radius: 10px;
          background-color: cornsilk;
          color: black;
          font-size: 14px;
        }
        </style>
    """))


# define a function that welcomes the user
# ask him how many questions to play
def main_menu(user):
  global total_questions
  global menu_text
  global page_container
  global logout_button
  global logout_hbox
  global i
  global score
  global correct_amount
  global info_button
  global session_btn
  global user_data
  global user_name
  global input_amount
  from google.colab.output import eval_js
  eval_js('google.colab.output.setIframeHeight("550")')

  user_data = user
  user_name = user["first_name"]
  score = i = input_amount = 0
  correct_amount = []

  logout_button = widgets.Button(description='Log Out', layout=widgets.Layout(width='75px'))#, button_style='primary')
  logout_button.add_class('logoutBtn_css')
  logged_in_user = widgets.HTML(value=f'<div><h3>Hello {user_name} ({user_data["points"]} points earned)</h3></div>')
  logout_hbox = widgets.HBox([logged_in_user, logout_button],
                             layout=widgets.Layout(justify_content='flex-end',
                                                   align_items='center'))
  #display(logout_hbox)
  logout_button.on_click(on_logout_clicked)

  #custom_styleInfo = {'button_color': 'cornsilk', 'color': 'black', 'border_color': 'black'}
  # Create a button widget
  info_button = widgets.Button(description='ⓘ', layout=widgets.Layout(width='50px'))#, style=custom_styleInfo)
  info_button.add_class('infoBtn_css')
  # Apply additional CSS classes to the button
  #info_button.button_style = 'info'
  # Set the button's tooltip
  info_button.tooltip = 'Click for information'

  # Display the button
  #display(info_button)
  info_button.on_click(lambda b: open_info())

  session_btn = widgets.Button(description="Your Profile", layout=widgets.Layout(width='120px'))
  session_btn.add_class('logoutBtn_css')

  session_btn.on_click(open_sessionGraph)
  info_profile_box = widgets.HBox([info_button, session_btn])
  session_btn.add_class("session_btn")

  menu_text = widgets.HTML(value=f'<div style="text-align:center;"><h2>How many questions would you like to play right now?</h2></div>')
  menu_text.value += f'<div style="text-align:center;"><h3>We have a total of {total_questions} questions</h3></div>'
  #display(menu_text)

  num_buttons = total_questions // 5
  newButton = total_questions % 5
  buttons = [widgets.Button(description=str(j * 5), button_style='primary') for j in range(1, num_buttons + 1)]
  if(newButton != 0):
    buttons.append(widgets.Button(description=str(total_questions), button_style='primary'))

  #for j, button in enumerate(buttons):
  #  button.add_class('buttons_css')

  for j, button in enumerate(buttons):
    if (j == num_buttons - 1) and (newButton != 0):
        button.on_click(lambda b: start_game(total_questions))
    else:
        button.on_click(lambda b, j=j: start_game((j + 1) * 5))


  # Divide the buttons into rows of two
  rows = [widgets.HBox(buttons[i:i+2]) for i in range(0, len(buttons), 2)]

  # Center the rows of buttons horizontally
  button_container = widgets.VBox(rows, layout=widgets.Layout(justify_content='center'))

  # Create a container for vertical centering
  page_container = widgets.VBox([button_container], layout=widgets.Layout(justify_content='center', align_items='center'))

  page_container = widgets.VBox([logout_hbox, info_profile_box, menu_text, page_container])
  # Display the rows of buttons
  display(page_container)

  page_container.add_class('body')

  display(HTML("""
        <style>
        .session_btn {
          margin-left: 10px
        }
        body {
            background-image: url('https://cutewallpaper.org/21/quiz-background/General-knowledge-quizzes-What-do-you-know.jpg');
            background-size: 1820px 566px;
          }
        .buttons_css {
          border-radius: 10px;
          background-color: dodgerblue;
        }
        .logoutBtn_css {
          border-radius: 10px;
          background-color: cornsilk;
          color: black;
        }
        .infoBtn_css {
          border-radius: 10px;
          background-color: cornsilk;
          color: black;
          font-size: 14px;
        }
        </style>
    """))



# define a function that gets an array of questions the player answered
# makes a sphere for each question
def display_correctness(correct_amount):
    spheres = []
    for correct in correct_amount:
        if correct == 1:
            color = 'green'
        else:
            color = 'red'
        sphere = f'<span style="display:inline-block;margin:5px;border-radius:50%;background-color:{color};width:25px;height:25px;"></span>'
        spheres.append(sphere)
    html = widgets.HTML(value=''.join(spheres))
    return html

def create_pie(correct_amount, totel_amount):
    incorrect_answers = totel_amount - correct_amount

    # Data for the pie chart
    data = [correct_amount, incorrect_answers]
    labels = ['Correct Answers', 'Incorrect Answers']
    colors = ['#66BB6A', '#EF5350']  # Green for correct, Red for incorrect

    # Create the pie chart
    pie = plt.pie(data, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)

    # Set the labels' color to white
    for label in pie[1]:
        label.set_color('white')

    # Set aspect ratio to be equal so that pie is drawn as a circle.
    plt.axis('equal')

    # Save the pie chart as an image file with transparent background
    with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as tmpfile:
        image_path = tmpfile.name
        plt.savefig(image_path, bbox_inches='tight', facecolor='none', transparent=True)


    # Close the figure to release memory
    plt.close()

    # Display the image inside a widgets.VBox
    image_widget = widgets.Image(value=open(image_path, 'rb').read(), format='png')
    return image_widget

# run menu -> which will start the game
#main_menu("Elad Fisher")


def openMenu():
  global user_data
  close_widgets()
  main_menu(user_data)

continue_exit_value = None
# define a function for continuing from current question to the next
def on_continue_clicked(b):
  global continue_exit_value
  global continue_button
  continue_exit_value = True
  continue_button.close()
  global question_text
  global answers_box
  global feedback_box
  global i
  global goodbye_text
  global input_amount
  global correct_amount
  global spheres_html
  global exit_button
  global result_box
  global score
  global user_data
  global session_id
  global pia_chart
  global question_counter

  if question_text != None:
    question_text.close()
  if question_counter != None:
    question_counter.close()
  if answers_box != None:
    answers_box.close()
  if feedback_box:
    feedback_box.close()

  # check if i answered the last question from range of input_amount
  i += 1
  if i < input_amount:
    # if not -> move to the next question
    ask_question(questions[i])
  else:
    # if yes -> display a text with the players score
    goodbye_text = widgets.HTML(f'<h3 style="color:ivory">Thank you for playing, you scored: {score} out of {input_amount}</h3>')

    # insert the session to DB
    session_obj = {session_id: {
        "created": datetime.datetime.now(),
        "score": score,
        "total": input_amount
         }}
    # calculating user total score
    result = FBconn.get('/Users/', None)
    for document_id, document_data in enumerate(result):
      if document_data["id"] == user_data["id"]:
        # Update the document in Firebase
        total_points = document_data["points"] + score
        total_sessions = document_data["sessions"]
        total_sessions.update(session_obj)
        # updating local user_data
        user_data["points"] = total_points
        user_data["sessions"] = total_sessions
        FBconn.put('/Users', document_id, user_data)

    refresh_exit_button()

    # call the function that creates a pia chart for current game
    pia_chart = create_pia(score, input_amount)
    if(input_amount <= 15):
      # call the function that makes spheres for each question and display them
      spheres_html = display_correctness(correct_amount)
      result_box = widgets.VBox([goodbye_text, pia_chart, spheres_html, exit_button],
                                layout=widgets.Layout(align_items='center',
                                                      justify_content='center'))
    else:
      result_box = widgets.VBox([goodbye_text, pia_chart, exit_button],
                          layout=widgets.Layout(align_items='center',
                                                justify_content='center'))
    display(result_box)
    result_box.add_class('body')

# define a function that checks and closes every global widget
def close_widgets():
  global exit_box
  global continue_button
  global goodbye_text
  global answers_box
  global question_text
  global feedback_box
  global spheres_html
  global menu_text
  global page_container
  global logout_hbox
  global result_box
  global session_btn
  global info_button
  global page_content
  global page_style
  global pia_chart

  if continue_button != None:
    continue_button.close()
  if exit_box != None:
    exit_box.close()
  if answers_box != None:
    answers_box.close()
  if question_text != None:
    question_text.close()
  if spheres_html != None:
    spheres_html.close()
  if feedback_box != None:
    feedback_box.close()
  if goodbye_text != None:
    goodbye_text.close()
  if menu_text != None:
    menu_text.close()
  if page_container != None:
    page_container.close()
  if logout_hbox != None:
    logout_hbox.close()
  if result_box != None:
    result_box.close()
  if info_button != None:
    info_button.close()
  if session_btn != None:
    session_btn.close()
  if pia_chart != None:
    pia_chart.close()
  clear_output()

# define a function for when you exit current game
def on_exit_clicked(b):
  global user_data
  global continue_exit_value
  continue_exit_value = False
  close_widgets()
  global goodbye_text
  goodbye_text = widgets.HTML('<h3 style="color:ivory">Thank you for playing</h3>')
  display(goodbye_text)
  time.sleep(3)
  goodbye_text.close()
  main_menu(user_data)

continue_button.on_click(on_continue_clicked)
exit_button.on_click(on_exit_clicked)
logout_button.on_click(on_logout_clicked)



HTML(value='\n  <style>\n  .exit_box {\n    /* padding: 35px */\n  }\n  </style>\n')

In [ ]:
import ipywidgets as widgets
from traitlets import Int

In [ ]:
#@title Update Question
import time
import requests

back_button = None
questions = FBconn.get('/Questions/', None)
form_output = None
form = None

def delete_question(index):
  global questions
  document_path = '/Questions/' + str(index)
  # Delete the document
  result = FBconn.delete(document_path, None)

  # Check if the deletion was successful
  if result is None:
      print("Document deleted successfully.")
  else:
      print("Failed to delete document.")
  time.sleep(1.2)
  questions = FBconn.get('/Questions/', None)
  clear_output()
  manager_Menu(name)

def update_question(question, index):
  global questions
  result = FBconn.get('/Questions/', None)
  for document_id, document_data in enumerate(result):
    if document_id == index:
      # Modify the document_data as needed
      updated_data = {
        'Question': question['Question'],
        'Option1': question['Option1'],
        'Option2': question['Option2'],
        'Option3': question['Option3'],
        'Option4': question['Option4'],
        'Correct': question['Correct']
      }
      # Update the document in Firebase
      FBconn.put('/Questions', str(index), updated_data)
      print("Update Question in Firebase.")
      time.sleep(2)
      questions = FBconn.get('/Questions/', None)
      clear_output()
      manager_Menu(name)
      break

def show_question(indexid):
  global form_items
  global form
  global form_output
  global index
  global back_button
  global image_widget
  # Create text input widgets
  index =  int(indexid[:-1])-1
  question_input = widgets.Textarea(description='Question:', rows=3, layout=widgets.Layout(width='400px'))
  question_input.value = questions[index]['Question']
  answer_a_input = widgets.Text(description='Option 1:', layout=widgets.Layout(width='400px'))
  answer_a_input.value = questions[index]['Option1']
  answer_b_input = widgets.Text(description='Option 2:', layout=widgets.Layout(width='400px'))
  answer_b_input.value = questions[index]['Option2']
  answer_c_input = widgets.Text(description='Option 3:', layout=widgets.Layout(width='400px'))
  answer_c_input.value = questions[index]['Option3']
  answer_d_input = widgets.Text(description='Option 4:', layout=widgets.Layout(width='400px'))
  answer_d_input.value = questions[index]['Option4']
  correct_answer_dropdown = widgets.Dropdown(
       options=['1', '2', '3', '4'],
      description='Correct:',
      layout=widgets.Layout(width='200px')
  )
  correct_answer_dropdown.value = str(questions[index]['Correct'])


  # Create an "Edit" button
  edit_button = widgets.Button(description='Update', button_style='warning')
  # Create an "Delete" button
  delete_button = widgets.Button(description='Delete', button_style='danger')
  delete_button.on_click(handle_delete_button)
  delete_button.add_class('button_css')
  # Create a form output widget
  form_output = widgets.Output()
  # Attach the function to the edit button click event
  edit_button.on_click(handle_edit_button_click)
  edit_button.add_class('button_css')
  # Create a form layout
  form_items = [
        widgets.HTML('<h2 style="font-weight:bold;text-align:center;">Edit question form</h2>'),
        widgets.HTML('<h4 style="font-weight:bold;"></h4>'),
        question_input,
        widgets.HTML('<h4 style="font-weight:bold;"></h4>'),
        answer_a_input,
        widgets.HTML('<h4 style="font-weight:bold;"></h4>'),
        answer_b_input,
        widgets.HTML('<h4 style="font-weight:bold;"></h4>'),
        answer_c_input,
        widgets.HTML('<h4 style="font-weight:bold;"></h4>'),
        answer_d_input,
        widgets.HTML('<h4 style="font-weight:bold;"></h4>'),
        correct_answer_dropdown,
        widgets.Label(value=''),
        edit_button,
        delete_button
  ]

  form = widgets.VBox(form_items, layout=widgets.Layout(width='auto', background_color='blue'))  # Add blue background color    height='300px'))
  form.add_class("form")
   # Create a wrapper container for centering the form
  form_wrapper = widgets.VBox([form], layout=widgets.Layout(justify_content='center', align_items='center', background_color='blue'))
  # Apply additional styling
  form_wrapper.add_class('centered-form')
  back_button = widgets.Button(description='Back to Menu', button_style='info')
  back_hbox = widgets.HBox([back_button],
                             layout=widgets.Layout(justify_content='flex-start',
                                                   align_items='center'))

  all_screen = widgets.VBox([form_wrapper, back_hbox])
  all_screen.add_class("all_screen")
  form_wrapper.add_class("form_wrapper")
  display(HTML("""
        <style>
        .form {
          backdrop-filter: blur(10px);
          border-radius: 15px;
        }
        .centered-form .widget-box {
            margin: auto;
            padding: 20px;
        }
        .centered-form .widget-label {
            font-weight: bold;
        }
        .button_css {
            font-weight: bold;
            margin-left: auto;
            margin-right: auto;
            border-radius: 10px;
        }
        .form_and_image_css {
            display: flex;
            justify-content: center;
            align-items: center;
        }
        .image_css {
            padding-left: 20px;  /* Adjust the padding value to create space between form and image */
        }
        </style>
        """))
  display(all_screen)

  back_button.on_click(back_on_click)


def update_page(m_name):
  global name
  global back_button
  global questions
  questions = FBconn.get('/Questions/', None)

  name = m_name
  list_question = []
  i = 1
  for q in questions:
    list_question.append(str(i) +'. ' + q['Question'])
    i+=1

  update_text = widgets.HTML(value=f'<div style="text-align:center;color:purple"><h2>Choose Question to update</h2></div>')


  question_dropdown = widgets.Dropdown(
      options= list_question,
      description='Questions:',
      layout=widgets.Layout(width='400px')
  )
  drop_layout = widgets.Layout(align_items='center', padding='20px', display='flex', flex_flow='column')
  dropBox_container = widgets.VBox([update_text, question_dropdown], layout=widgets.Layout(justify_content='center', align_items='center'))
  logout_button = widgets.Button(description='Log Out', button_style='primary')
  logout_hbox = widgets.HBox([logout_button],
                             layout=widgets.Layout(justify_content='flex-end',
                                                   align_items='center'))
  # display(logout_hbox)
  logout_button.on_click(on_logout_clicked)

  # Display the form
  # display(dropBox_container)
  # Assign the function to the dropdown's value change event
  question_dropdown.observe(handle_dropdown_change, names='value')
  # display(back_hbox)
  all_screen = widgets.VBox([logout_hbox, dropBox_container])
  all_screen.add_class("all_screen")

  display(HTML("""
        <style>
        .all_screen {
            background-image: linear-gradient(rgba(0, 0, 0, 0.327),rgba(0, 0, 0, 0.3)) , url(https://img.freepik.com/free-vector/blue-web-support-help-page-template-design_1017-26768.jpg);
            background-size: 1820px 640px;
        }
        </style>
        """))

  display(all_screen)
  handle_dropdown_change("first")


# Define a function to handle the dropdown value change event
def handle_dropdown_change(change):
    global back_button
    global form_output
    global form
    if back_button:
      back_button.close()
    if form_output != None:
      form_output.close()
    if form != None:
      form.close()

    if change == "first":
      show_question('1.')
    else:
      selected_question = change.new
      id_question = selected_question.split(' ')
      show_question(id_question[0])

def handle_delete_button(button):
  global index
  delete_question(index)

# Create a function to handle the edit button click event
def handle_edit_button_click(button):
  global form_items
  question = form_items[2].value
  answer_a = form_items[4].value
  answer_b = form_items[6].value
  answer_c = form_items[8].value
  answer_d = form_items[10].value
  correct_answer = form_items[12].value
  # Store the form values in an array
  form_values = {
      'Question': question,
      'Option1': answer_a,
      'Option2': answer_b,
      'Option3': answer_c,
      'Option4': answer_d,
      'Correct': int(correct_answer)
    }
  # Perform further processing or submit the form
  update_question(form_values, index)


In [ ]:
#@title Manager
import ipywidgets as widgets
from traitlets import Int
from IPython.display import HTML,display
from bs4 import BeautifulSoup
from ipywidgets import HTML, RadioButtons, VBox, HBox, Layout
import time
import requests

# Define a function to ask a question
def insert_question(question):
  questions = FBconn.get('/Questions/', None)
  questionId = len(questions)
  result = FBconn.put('/Questions/', questionId, question)

  if result:
    print("JSON object inserted into Firebase.")
    time.sleep(1.2)
    clear_output()
    manager_Menu(name)
  else:
    print("Failed to insert JSON object into Firebase.")

def back_on_click(b):
  clear_output()
  manager_Menu(name)

# Insert button click event handler
def insert_button_clicked(b):
    global form_items
    global error_output
    clear_output()
    # Define the form fields
    question_input = widgets.Textarea(description='Question:', rows=2, layout=widgets.Layout(width='300px'))
    answer_a_input = widgets.Text(description='Option 1:', layout=widgets.Layout(width='300px'))
    answer_b_input = widgets.Text(description='Option 2:', layout=widgets.Layout(width='300px'))
    answer_c_input = widgets.Text(description='Option 3:', layout=widgets.Layout(width='300px'))
    answer_d_input = widgets.Text(description='Option 4:', layout=widgets.Layout(width='300px'))
    correct_answer_dropdown = widgets.Dropdown(
        options=['1', '2', '3', '4'],
        description='Correct:',
        layout=widgets.Layout(width='300px')
    )
    insert_text = widgets.HTML(value=f'<div style="text-align:center;color:purple"><h2>Fill your Question</h2></div>')
    # Create a submit button
    submit_button = widgets.Button(description='Submit', button_style='success')
    error_output = widgets.Output()
    # Assign the form submission function to the button click event
    submit_button.on_click(submit_form)
    submit_button.add_class('button_css')
    # Display the form
    form_items = [
        widgets.HTML('<h2 style="font-weight:bold;text-align:center;">New question form</h2>'),
        widgets.HTML('<h4 style="font-weight:bold;"></h4>'),
        question_input,
        widgets.HTML('<h4 style="font-weight:bold;"></h4>'),
        answer_a_input,
        widgets.HTML('<h4 style="font-weight:bold;"></h4>'),
        answer_b_input,
        widgets.HTML('<h4 style="font-weight:bold;"></h4>'),
        answer_c_input,
        widgets.HTML('<h4 style="font-weight:bold;"></h4>'),
        answer_d_input,
        widgets.HTML('<h4 style="font-weight:bold;"></h4>'),
        correct_answer_dropdown,
        widgets.Label(value=''),
        submit_button,
        error_output
    ]
    form = widgets.VBox(form_items, layout=widgets.Layout(width='400px', background_color='blue'))  # Add blue background color    height='300px'))
    form.add_class("form")
    # Create a wrapper container for centering the form
    form_wrapper = widgets.VBox([form], layout=widgets.Layout(justify_content='center', align_items='center', background_color='blue'))
    logout_button = widgets.Button(description='Log Out', button_style='primary')
    logout_hbox = widgets.HBox([logout_button],
                             layout=widgets.Layout(justify_content='flex-end',
                                                   align_items='center'))
    logout_button.on_click(on_logout_clicked)
    # Apply additional styling
    form_wrapper.add_class('centered-form')

    back_button = widgets.Button(description='Back to Menu', button_style='info')
    back_hbox = widgets.HBox([back_button],
                             layout=widgets.Layout(justify_content='flex-start',
                                                   align_items='center'))

    all_screen = widgets.VBox([logout_hbox, form_wrapper, back_hbox])
    all_screen.add_class('all_screen')

    display(HTML("""
        <style>
        .form {
          backdrop-filter: blur(10px);
        }
        .all_screen {
            background-image: url('https://e0.pxfuel.com/wallpapers/102/888/desktop-wallpaper-3d-question-marks-figures-cool-question-mark.jpg');
            background-size: 1820px 800px;
        }
        .centered-form .widget-box {
            margin: auto;
            border: 2px solid black;
            padding: 20px;
        }
        .centered-form .widget-label {
            font-weight: bold;
        }
        .button_css {
            font-weight: bold;
            margin-left: auto;
            margin-right: auto;
            border-radius: 10px;
        }
        .form_and_image_css {
            display: flex;
            justify-content: center;
            align-items: center;
        }
        </style>
        """))
    # Display the form
    display(all_screen)
    back_button.on_click(back_on_click)

# Update button click event handler
def update_button_clicked(b):
    clear_output()
    update_page(name)



# Create a function to handle the form submission
def submit_form(button):
    global error_output
    error_output.clear_output()
    error_messages = []
    # Retrieve the form values
    question = form_items[2].value
    answer_a = form_items[4].value
    answer_b = form_items[6].value
    answer_c = form_items[8].value
    answer_d = form_items[10].value
    correct_answer = form_items[12].value

    # Check if any field is left empty
    if not question:
        error_messages.append("Question field is required.")
    if not answer_a:
        error_messages.append("Option 1 field is required.")
    if not answer_b:
        error_messages.append("Option 2 field is required.")
    if not answer_c:
        error_messages.append("Option 3 field is required.")
    if not answer_d:
        error_messages.append("Option 4 field is required.")
    if not correct_answer:
        error_messages.append("Correct Answer field is required.")

    # Display error messages if any
    if error_messages:
        with error_output:
            error_output.clear_output()
            error_html = '<div style="text-align: center; color: red;">' + '<br>'.join(error_messages) + '</div>'
            display(HTML(error_html))
    else:
        # Store the form values in an array
        form_values = {
            'Question': question,
            'Option1': answer_a,
            'Option2': answer_b,
            'Option3': answer_c,
            'Option4': answer_d,
            'Correct': int(correct_answer)
        }

        # Perform further processing or submit the form
        insert_question(form_values)

def manager_Menu(user_name):
  global name
  name = user_name
  page_style = f"<style>.jp-Notebook {{ background-color: #f0f0f0; }}</style>"
  display(HTML(page_style))
  # Create menu text
  menu_text = widgets.HTML(value=f'<div style="text-align:center;"><h1>Hello {user_name}</h1></div>')
  menu_text.value += f'<div style="text-align:center;"></div>'


  # Create menu buttons with custom styling
  insert_button = widgets.Button(description='Insert Question', layout=widgets.Layout(width='200px', height='50px'))
  insert_button.style.button_color = '#03a16a'
  insert_button.style.font_weight = 'bold'
  insert_button.style.font_color = 'black'

  update_button = widgets.Button(description='Update/Delete Question', layout=widgets.Layout(width='200px', height='50px'))
  update_button.style.button_color = '#be4d25'
  update_button.style.font_weight = 'bold'
  update_button.style.font_color = 'black'
  # Register click event handlers for the buttons
  insert_button.on_click(insert_button_clicked)
  update_button.on_click(update_button_clicked)
  logout_button = widgets.Button(description='Log Out', button_style='primary')
  logout_hbox = widgets.HBox([logout_button],
                             layout=widgets.Layout(justify_content='flex-end',
                                                   align_items='center'))
  # display(logout_hbox)
  logout_button.on_click(on_logout_clicked)
  # Set the background color of the page
  # display(menu_text)

  # Display the buttons
  button_container = widgets.HBox([insert_button, update_button], layout=widgets.Layout(justify_content='center'))
  all_screen = widgets.VBox([logout_hbox, menu_text, button_container])
  all_screen.add_class("all_screen")

  display(HTML("""
      <style>
      .all_screen {
          background-image: url('https://img.freepik.com/free-vector/half-tone-blue-abstract-background-with-text-space_1017-41428.jpg');
          background-size: 1820px 140px;
      }
      </style>
      """))

  display(all_screen)
  # display(button_container)


In [ ]:
#@title Login Screen


from IPython.display import display, HTML, clear_output
import google.colab.output
from firebase import firebase
##imports and drive connect
import IPython
from google.colab import drive

from IPython.display import HTML

def get_user(name,password):
      res=FBconn.get('/Users/',None)
      for user in res:
        if user['first_name'] == name and user['password'] == password:
            return user
      return False

def close_error():
  clear_output()
  display(HTML(template))

def LoginFunc(username, password):
  if username == "":
  # display(HTML(error))
      print("Please enter a username")
      return

  if password == "":
    #  display(HTML(error))
    print("Please enter a password")
    return
  user_from_db=get_user(username,password)
  if user_from_db:
    if user_from_db['role'] == 'customer':
      rule = 'customer'
      clear_output()
      main_menu(user_from_db)

    else:
      clear_output()
      manager_Menu(user_from_db['first_name'])
  else:
    print("Username or password is inncorrect! Please try again")

def Login():
        error="""
        <!DOCTYPE html>
        <html lang="en">
        <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Error Page</title>
        <style>
        body {
        display: flex;
        justify-content: center;
        align-items: center;
        height: 100vh;
        font-family: Arial, sans-serif;
        }
        .error-box {
        background-color: #fff;
        border-radius: 8px;
        padding: 20px;
        box-shadow: 0 2px 5px rgba(0, 0, 0, 0.2);
        }
        h1 {
        font-size: 24px;
        margin: 0 0 20px;
        text-align: center;
        }
        p {
        font-size: 16px;
        margin: 0 0 20px;
        text-align: center;
        }
        .error-message {
        color: red;
        font-weight: bold;
        }
        .close-button {
        background-color: #ccc;
        color: #fff;
        border: none;
        border-radius: 4px;
        padding: 10px 20px;
        font-size: 16px;
        font-weight: bold;
        cursor: pointer;
        }
        </style>
        </head>
        <body>
        <div class="error-box">
        <h1>Error</h1>
        <p class="error-message">The username or password is incorrect. Please try again.</p>
        <button class="close-button">Close</button>
        </div>

        <script>
        const closeButton = document.querySelector('.close-button');
        closeButton.addEventListener('click', () => {
        // Close the error box or perform any desired action
        google.colab.kernel.invokeFunction(
          "errorfunc",
          [],
          {}
        );
        });
        </script>
        </body>
        </html>
        """
        template = """

            <!DOCTYPE html>
            <html lang="en">
            <head>
              <meta charset="UTF-8" />
              <link rel="icon" type="image/svg+xml" href="/vite.svg" />
              <meta name="viewport" content="width=device-width, initial-scale=1.0" />
            </head>
            <body>
            <style>
              body {
                display: flex;
                justify-content: center;
                background-image: url('https://t3.ftcdn.net/jpg/04/48/06/74/360_F_448067440_hTWoSx63cvavKS0qYDs1BR2Qg5C0JVB7.jpg');
                background-size: 1820px 520px;
                height: 450px;
                margin: 0;
              }
              h1 {
                font-family: times, Times New Roman, times-roman, georgia, serif;
                color: white;
                font-size: 51px;
                line-height: 44px;
                letter-spacing: -2px;
                font-weight: bold;
              }
              .python-logo {
                width: 50px;
                height: 50px;
                margin-right: 10px;
              }
              .img-div{
                padding:70px 0 0 10px;
              }
              .img-connect {
                width:35%;
                height: 45%;
                margin-right: 10px;
              }
              .information-container {
                display: flex;
                align-items: flex-start; /* Updated alignment */
              }
              .login-form {
                margin-bottom: 20px;
                margin: 70px 0px 0px 427px;
                background: #fff;
                width: 350px;
                height: auto;
                display: flex;
                flex-direction: column;
                align-items: center;
                justify-content: space-around;
                border-radius: 8px;
                padding: 15px;
                box-shadow: -10px 14px 6px 0px #ccc;
              }
              .form-group {
                width: 80%;
                align-self: center;
                margin-bottom: 15px;
              }
              .form-control {
                width: 100%;
                padding: 10px;
                border: 1px solid #ccc;
                border-radius: 4px;
              }
              .btn {
                width: 108%;
                padding: 10px;
                border: none;
                border-radius: 4px;
                font-size: 16px;
                font-weight: bold;
                cursor: pointer;
              }
              .btn-primary {
                background-color: #007bff;
                color: #fff;
              }
              .btn-secondary {
                background-color: #6c757d;
                color: #fff;
              }
              .btn:hover {
                opacity: 0.8;
              }
            </style>

            <h1>
                &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
              <img src="https://upload.wikimedia.org/wikipedia/commons/c/c3/Python-logo-notext.svg" alt="Python Logo" class="python-logo">
              Python Game
            </h1>
            <div class="information-container">
            <div class="login-form">
              <div class="form-group"   >
                <input
                  type="text"
                  name="username"
                  id="username"
                  placeholder="Username"
                  class="form-control"
                />
              </div>
              <div class="form-group">
                <input
                  type="password"
                  name="password"
                  id="password"
                  placeholder="Password"
                  class="form-control"
                />
              </div>
              <div class="form-group">
                <button class="btn btn-primary" onclick="logfunc()">login</button>
              </div>
            </div>

                 <div class="img-div">
                 <img src="https://cdn2.iconfinder.com/data/icons/web-developer-1/60/Female-Developer-avatar-person-user-man-programmer-512.png" alt="user-6318011-1920"  class="img-connect">
                  </div>
           </div>
            <script>

                    function logfunc() {
                    {
                    let username = document.querySelector("#username").value;
                    let password = document.querySelector("#password").value;


                    google.colab.kernel.invokeFunction(
                      "logfunc",
                      [username, password],
                      {}
                    );
                    }
                    }
                    </script>
                    </body>
                    </html>

                """


        display(HTML(template))



#clear_output()
google.colab.output.register_callback('logfunc', LoginFunc)

google.colab.output.register_callback('errorfunc', close_error)

Login();

HTML(value='<h3 style="color: lightgreen">Returning to log-in screen shortly</h3>')